# GoogLeNet V3

论文:[Rethinking the Inception Architecture for Computer Vision](https://arxiv.org/abs/1512.00567)

GoogLeNet的一个重要改进是分解(Factorization)，将较大的卷积核分解为用多个较小的卷积核来代替，这样的好处是较小的卷积核可以加速计算，同时，多个小卷积核的使用使得网络层数增加，从而间接提高了网络的非线性能力，另外就是将输入的尺寸变大，并更加精细的重新设计了某些模块。

## 网络设计原则

文中中首先提出一些在实验中证明过的设计网络时应该遵循的原则：

* 1.避免表达表达瓶颈，特别是在网络靠前的地方，信息流前向传播的过程中显然不能经过高度压缩的层，即表达瓶颈，从输入到输出，$feature map$的宽和高基本都会逐渐变小，但是不能一下子就变得很小，比如上来就是$kernel=7，stride=5$，这显然不合适。另外，输出的维度channel，一般来说会逐渐增多，否则网络会很难训练，同时特征的维度并不代表信息的多少(因为还要考虑信息之间的相关性)，他只是一种估计信息量的手段而已。

    + **另一种说法：避免表达的瓶颈，表达的尺寸（即feature map的大小）不应该出现急剧的衰减，如果对流经层（尤其是卷积层）的信息过度的压 缩，将会丢失大量的信息，对模型的训练也造成了困难。**

* 2.高维特征特征更容易处理，高维特征更容易区分，因此会加快训练。
    
    + **另一种说法：在网络中对高维的表达进行局部的处理，将会使网络的训练增快。**

* 3.可以在低维空间上进行空间汇聚而无需担心丢失很多信息。比如，在进行3X3的卷积前，可以对输入先进行降维而不会产生严重的后果。假设信息可以被简单压缩，那么训练就会加快。

    + **另一种说法：在较低维度的输入上进行空间聚合，将不会造成任何表达能力上的损失，因为feature map上，临近区域上的表达具有很高的相关性，如果对输出进行空间聚合，那么将feature map的维度降低也不会减少表达的信息。这样的话，有利于信息的压缩，并加快了训练的速度。**

* 4.设计网络的深度和宽度达到一个平衡的状态，要使得计算资源平衡的分配在模型的深度和宽度上面，才能最大化的提高模型的性能。

## 改进方案

#### 1.Factorizing Convolutions with Large Filter Size 

考虑到第三条设计准则，由于feature map的临近区域具有很高的相关性，再使用空间聚合的前提下，可以减小feature map也就是activation的维度，因此也可以使用更小的滤波器（卷积）来提取局部的特征，同时这样做还可以提高计算的效率，例如将$\7 \times 7$的卷积替换成3个$3 \times 3$卷积的叠加，将$5 \times 5$的卷积替换成2个$3 \times 3$卷积的叠加，这也是VGG所提到的改进。 

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-6.png" width="400">

这在参数量和计算量上都减少了很多。多层的表达能力不会受到影响，同时增加了非线性修正，对模型的效果进行了改善。

GoogLeNet V1中的Inception模块：

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-9.png" width="400">

将$5 \times 5$卷积核替换后的Inception模块：

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-10.png" width="400">

那么参数量和计算量可否进一步的减少呢？作者首先想到的是将$3 \times 3$的卷积核分解为两个$2 \times 2$的卷积核，但计算量只减少了$11\%$，因此作者又提出了一种非对称的方式，用$3 \times 1$的卷积核并接上一个$1 \times 3$的卷积核来等效一个$3 \times 3$的卷积核， 这样不仅减少了参数量，在计算量上又降低了33%，下图为计算示意图。

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-7.png" width="400">

随之作者认为，理论上任何$n \times n$的卷积核都可以用两个$n \times 1$和$1 \times n$的卷积核来代替，且随着$n$的增加，计算量减少的效果会更为显著。但在实践中，作者发现，由于Inception的特性，在前面的层使用这种非对称卷积方式效果并不是很好，但去在中等尺寸的$m \times m$的feature map上取得了较为理想的效果，其中$12 \le m \le 20$，尤其是当使用$1 \times 7$和$7 \times 1$的非对称卷积时的效果最好。

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-11.png" width="400">

#### 2.Utility of Auxiliary Classifiers 

在GoogLeNet的原始论文中提到了，在模型的中间层上使用了辅助的分类器，因为作者认为中间层的特征将有利于提高最终层的判别力。但是在这篇文章中，作者发现辅助层在训练初期并没有起到很好的效果，只有在增加了BN层或者dropout时，能够提升主分类器的性能。

#### 3.Efficient Grid Size Reduction 

为了避免第一条准则中提到的计算瓶颈所造成的的信息损失，一般会通过增加滤波器的数量来保持表达能力，但是计算量会增加。作者提出了一种并行的结构，使用两个并行的步长为2的模块，P和C。P是一个池化层，C是一个卷积层，然后将两个模型的响应组合到一起，这样做既没有造成信息提取上的损失，也能够很好的降低维度，同时减少了计算代价。

作者认为下图的两种方式(即若想从$d \times d \times k$变为$\frac{d}{2} \times \frac{d}{2} \times 2k$)都存在问题，左面的方式会引入表达瓶颈，即信息流的高度压缩会减弱模型的学习能力，而右面的方式会带来额外的计算代价。

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-12.png" width="400">

因此作者提出了一种并行的方式来改善这种问题，下图是这种并行方式。

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-13.png" width="400">

#### 4.针对低分辨率输入的改进

作者还提出了针对低分辨率的解决方案，因为机器所要识别的目标并不总是高分辨率的，有可能是比较小，或者分辨率比较低的。 

常用的做法是使用更高分辨率的接受域，但是有时候我们需要分清，究竟是使用更高分辨率接受域的效果，还是更大的模型容量和计算量的效果。 

作者提出的方法如下：针对低分辨率的图像输入，应该降低前两层的步长，或移除第一个池化层(One simple way to ensure constant effort is to reduce the strides of the first two layer in the case of lower resolution input, or by simply removing the first pooling layer of the network)。

#### 说明：$5 \times 5$卷积核替换为两个级联的$3 \times 3$卷积核在参数和计算量上的影响

文章中提到了，用两个$3 \times 3$的卷积核可以等效为一个$5 \times 5$卷积核的效果，用三个$3 \times 3$的卷积核可以等效为一个$7 \times 7$卷积核的效果，同时减少了参数量。

大尺寸的卷积核可以带来更大的感受野，但也意味着更多的参数，比如$5 \times 5$卷积核参数(仅考虑w不考虑b)是$3 \times 3$卷积核的$（5\times5+1）/（3\times3+1）=2.6$倍

## GoogLeNet V3 - Inception V3

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-8.png" width="450">

## 总结

Inception V3设计了三种形式的Inception模块，包括将**大滤波器进行分解、分解成不对称结构、以及通过并行结构**来解决信息损失的问题，虽然模型的拓扑结构更加复杂了，但是计算量大大降低，而且模型性能大大提高了，总的来看，今后的模型主要解决两个问题：

1.如何在计算量不增加的情况下，解决由于信息压缩造成的信息损失问题。

2.如果在计算量不增加的情况下，增加模型的拓扑结构，以提高模型的表达能力。

## 参考文献

1.[5*5卷积核替换为两个级联3*3卷积核在参数和计算量上的影响](https://zhuanlan.zhihu.com/p/25431979)

2.[GoogLeNet：Inception V3：Rethinking the Inception Architecture for Computer Vision论文笔记](http://blog.csdn.net/wspba/article/details/68065564)

In [1]:
import mxnet as mx

from mxnet import nd
from mxnet import gluon
from mxnet import autograd

import utils
mx.random.seed(1)

ctx = mx.gpu()

In [2]:
# Conv_BN_ReLU
class Conv_BN_ReLU(gluon.Block):
    def __init__(self, channels, kernel_size, strides=1, padding=0, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.Conv2D = gluon.nn.Conv2D(channels, kernel_size=kernel_size, strides=strides, padding=padding)
        self.BN = gluon.nn.BatchNorm(axis=1)
        
    def forward(self, X):
        out = nd.relu(self.BN(self.Conv2D(X)))
        if self.debug:
            print("Conv_BN_ReLU : ", out.shape)
        return out

In [3]:
# test Conv_BN_ReLU
cbr = Conv_BN_ReLU(32, 3, strides=2, debug=True)
cbr.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32, 3, 299, 299), ctx=ctx)
y = cbr(X)

Conv_BN_ReLU :  (32, 32, 149, 149)


In [4]:
cbr = Conv_BN_ReLU(32, kernel_size=(1,3), strides=1, debug=True)
cbr.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32, 288, 35, 35), ctx=ctx)
y = cbr(X)

Conv_BN_ReLU :  (32, 32, 35, 33)


In [5]:
cbr = Conv_BN_ReLU(32, kernel_size=(3,1), strides=1, debug=True)
cbr.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32, 32, 35, 33), ctx=ctx)
y = cbr(X)

Conv_BN_ReLU :  (32, 32, 33, 33)


In [6]:
cbr = Conv_BN_ReLU(32, kernel_size=(3,3), strides=1, debug=True)
cbr.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32, 288, 35, 35), ctx=ctx)
y = cbr(X)

Conv_BN_ReLU :  (32, 32, 33, 33)


### 实现论文中图5的Inception

In [7]:
# Figure 5 From the paper 
class Inception5(gluon.Block):
    def __init__(self, n1_1, n1_double_3, n2_1, n2_3, n3_pool_1, n3_1, n4_1, 
                 debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        # path1
        self.p1_1 = Conv_BN_ReLU(n1_1, kernel_size=1)
        self.p1_3_1 = Conv_BN_ReLU(n1_double_3, kernel_size=3, padding=1)
        self.p1_3_2 = Conv_BN_ReLU(n1_double_3, kernel_size=3, padding=1)
        # path2 
        self.p2_1 = Conv_BN_ReLU(n2_1, kernel_size=1)
        self.p2_3 = Conv_BN_ReLU(n2_3, kernel_size=3, padding=1)
        # path3
        self.p3_pool_1 = gluon.nn.AvgPool2D(pool_size=3, strides=1, padding=1)
        self.p3_1 = Conv_BN_ReLU(n3_1, kernel_size=1)
        # path4
        self.p4_1 = Conv_BN_ReLU(n4_1, kernel_size=1)
        
    def forward(self, X):
        p1 = self.p1_1(self.p1_3_1(self.p1_3_2(X)))
        p2 = self.p2_1(self.p2_3(X))
        p3 = self.p3_pool_1(self.p3_1(X))
        p4 = self.p4_1(X)
        out = nd.concat(p1, p2, p3, p4, dim=1)
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("p3 : ", p3.shape)
            print("p4 : ", p4.shape) 
        return out

In [8]:
incep5 = Inception5(64, 48, 64, 64, 96, 96, 32, debug=True)
incep5.initialize(ctx=ctx)
X = nd.random.uniform(shape=(32, 192, 35, 35), ctx=ctx)
y = incep5(X)
y.shape 

p1 :  (32, 64, 35, 35)
p2 :  (32, 64, 35, 35)
p3 :  (32, 96, 35, 35)
p4 :  (32, 32, 35, 35)


(32, 256, 35, 35)

#### 参考CNTK实现中发现每个阶段中间还夹杂了一个Inception模块，该模块的结果是三条路径的汇合：

#### Path 1
* $3 \times 3 Conv, strides=2, padding=0;$

#### Path 2
* $1 \times 1 Conv, strides=1, padding=0;$
* $3 \times 3 Conv, strides=1, padding=1;$
* $3 \times 3 Conv, strides=2, padding=0;$

####  Path 3
* $Maxpooling, pool\_size=3, strides=2, padding=0;$

虽然不知道为什么有这个实现模块，但是我们还是来实现一下吧。但是该模块的主要作用好像是feature map减半

In [9]:
class Inception5_1(gluon.Block):
    def __init__(self, n1_1, n2_1, n2_double_3, n3_1, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_1 = Conv_BN_ReLU(n1_1, kernel_size=3, strides=2)
        self.p2_1 = Conv_BN_ReLU(n2_1, kernel_size=1)
        self.p2_3_1 = Conv_BN_ReLU(n2_double_3, kernel_size=3, padding=1)
        self.p2_3_2 = Conv_BN_ReLU(n2_double_3, kernel_size=3, strides=2)
        self.p3_1 = gluon.nn.MaxPool2D(pool_size=3, strides=2)
       
    def forward(self, X):
        p1 = self.p1_1(X)
        p2 = self.p2_3_2(self.p2_3_1(self.p2_1(X)))
        p3 = self.p3_1(X)
        out = nd.concat(p1, p2, p3, dim=1)
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("p3 : ", p3.shape)
        return out

In [10]:
incep51 = Inception5_1(384, 64, 96, 96, debug=True)
incep51.initialize(ctx=ctx)
X = nd.random.uniform(shape=(1, 288, 35, 35), ctx=ctx)
y = incep51(X)
y.shape 

p1 :  (1, 384, 17, 17)
p2 :  (1, 96, 17, 17)
p3 :  (1, 288, 17, 17)


(1, 768, 17, 17)

### 实现论文中图6的Inception

我们首先来验证下$n \times n$的卷积可以用$1 \times n$和$n \times 1$两个模块来代替

注意这个结论只有当$Same \, padding$的时候才成立

In [11]:
print("------1×n and n×1 conv------")
cbr1 = Conv_BN_ReLU(32, kernel_size=(1,7), strides=1, padding=(0,3))
cbr1.initialize(ctx=ctx)
X = nd.random.uniform(shape=(1, 768, 17, 17), ctx=ctx)
print('input : ', X.shape)
y = cbr1(X)
print('1 X 3 Conv : ', y.shape)

cbr2 = Conv_BN_ReLU(32, kernel_size=(7,1), strides=1, padding=(3,0))
cbr2.initialize(ctx=ctx)
z = cbr2(y)
print('3 X 1 Conv : ', z.shape)

print("------nxn conv------") 
cbr3 = Conv_BN_ReLU(32, kernel_size=(7,7), strides=1, padding=3)
cbr3.initialize(ctx=ctx)
yy = cbr3(X)
print('3 X 3 Conv : ', yy.shape)

------1×n and n×1 conv------
input :  (1, 768, 17, 17)
1 X 3 Conv :  (1, 32, 17, 17)
3 X 1 Conv :  (1, 32, 17, 17)
------nxn conv------
3 X 3 Conv :  (1, 32, 17, 17)


In [12]:
# Figure 6 from the paper 
# 论文中使用的是n=7，因为作者验证了当feature map为17X17时，使用n=7的效果最好
# 因为这里参数太多，因此我们改用list来盛放参数
class Inception6(gluon.Block):
    def __init__(self, k, n1, n2, n3, n4, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        # path 1
        self.p1_1 = Conv_BN_ReLU(n1[0], kernel_size=1)
        ######### 此处表示same padding 此处的padding要计算得来
        self.p1_2 = Conv_BN_ReLU(n1[1], kernel_size=(1,k), padding=(0,3))
        self.p1_3 = Conv_BN_ReLU(n1[2], kernel_size=(k,1), padding=(3,0))
        self.p1_4 = Conv_BN_ReLU(n1[3], kernel_size=(1,k), padding=(0,3))
        self.p1_5 = Conv_BN_ReLU(n1[4], kernel_size=(k,1), padding=(3,0))
        # path 2
        self.p2_1 = Conv_BN_ReLU(n2[0], kernel_size=1)
        ######### 此处表示same padding 此处的padding要计算得来
        self.p2_2 = Conv_BN_ReLU(n2[1], kernel_size=(1,k), padding=(0,3))
        self.p2_3 = Conv_BN_ReLU(n2[2], kernel_size=(k,1), padding=(3,0))
        # path 3
        self.p3_1 = gluon.nn.AvgPool2D(pool_size=3, strides=1, padding=1)
        self.p3_2 = Conv_BN_ReLU(n3, kernel_size=1)
        # path 4
        self.p4_1 = Conv_BN_ReLU(n4, kernel_size=1) 
        
    def forward(self, X):
        p1 = self.p1_5(self.p1_4(self.p1_3(self.p1_2(self.p1_1(X)))))
        p2 = self.p2_3(self.p2_2(self.p2_1(X)))
        p3 = self.p3_2(self.p3_1(X))
        p4 = self.p4_1(X)
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("p3 : ", p3.shape)
            print("p4 : ", p4.shape)
        out = nd.concat(p1, p2, p3, p4, dim=1)
        return out

In [13]:
incep6 = Inception6(7, [128, 128, 128, 128, 192], [128, 128, 192], 192, 192, debug=True)
incep6.initialize(ctx=ctx)
X = nd.random.uniform(shape=(1, 768, 17, 17), ctx=ctx)
y = incep6(X)
y.shape 

p1 :  (1, 192, 17, 17)
p2 :  (1, 192, 17, 17)
p3 :  (1, 192, 17, 17)
p4 :  (1, 192, 17, 17)


(1, 768, 17, 17)

#### 参考CNTK实现中同样发现夹杂了一个Inception模块，该模块的结果是三条路径的汇合（与上面的不同）：

#### Path 1 (3X3)
* $1 \times 1 Conv, strides=1, padding=0;$
* $3 \times 3 Conv, strides=2, padding=0;$

#### Path 2  (7X7, 3X3)
* $1 \times 1 Conv, strides=1, padding=0;$
* $1 \times 7 Conv, strides=1, padding=(0,3);$
* $3 \times 7 Conv, strides=1, padding=(3,0);$
* $3 \times 3 Conv, strides=2, padding=0;$

####  Path 3 (Maxpooling)
* $Maxpooling, pool\_size=3, strides=2, padding=0;$

虽然不知道为什么有这个实现模块，但是我们还是来实现一下吧。但是该模块的主要作用好像是feature map减半

In [14]:
class Inception6_1(gluon.Block):
    def __init__(self, k, n1, n2, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        self.p1_1 = Conv_BN_ReLU(n1[0], kernel_size=1)
        self.p1_2 = Conv_BN_ReLU(n1[1], kernel_size=3, strides=2)
        
        self.p2_1 = Conv_BN_ReLU(n2[0], kernel_size=1)
        self.p2_2 = Conv_BN_ReLU(n2[1], kernel_size=(1,k), padding=(0,3))
        self.p2_3 = Conv_BN_ReLU(n2[2], kernel_size=(k,1), padding=(3,0))
        self.p2_4 = Conv_BN_ReLU(n2[3], kernel_size=3, strides=2)
    
        self.p3_1 = gluon.nn.MaxPool2D(pool_size=3, strides=2)
       
    def forward(self, X):
        p1 = self.p1_2(self.p1_1(X))
        p2 = self.p2_4(self.p2_3(self.p2_2(self.p2_1(X))))
        p3 = self.p3_1(X)
        out = nd.concat(p1, p2, p3, dim=1)
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("p3 : ", p3.shape)
        return out

In [15]:
incep61 = Inception6_1(7, [192, 320], [192, 192, 192, 192], debug=True)
incep61.initialize(ctx=ctx)
X = nd.random.uniform(shape=(1, 768, 17, 17), ctx=ctx)
y = incep61(X)
y.shape 

p1 :  (1, 320, 8, 8)
p2 :  (1, 192, 8, 8)
p3 :  (1, 768, 8, 8)


(1, 1280, 8, 8)

### 实现论文中图7的Inception

In [16]:
# figure 7 from paper 
class Inception7(gluon.Block):
    def __init__(self, n1, n2, n3, n4, debug=False, **kwargs):
        super().__init__(**kwargs)
        self.debug = debug
        # path1
        self.p1_1 = Conv_BN_ReLU(n1[0], kernel_size=1)
        self.p1_2 = Conv_BN_ReLU(n1[1], kernel_size=3, padding=1)
        self.p1_3_1 = Conv_BN_ReLU(n1[2][0], kernel_size=(1,3), padding=(0,1))
        self.p1_3_2 = Conv_BN_ReLU(n1[2][1], kernel_size=(3,1), padding=(1,0))
#         self.p1_3 = nd.concat(self.p1_3_1, self.p1_3_2, dim=1)
        # path2
        self.p2_1 = Conv_BN_ReLU(n2[0], kernel_size=1)
        self.p2_2_1 = Conv_BN_ReLU(n2[1][0], kernel_size=(1,3), padding=(0,1))
        self.p2_2_2 = Conv_BN_ReLU(n2[1][1], kernel_size=(3,1), padding=(1,0))
#         self.p2_2 = nd.concat(self.p2_2_1, self.p2_2_2, dim=1)
        # path3
        self.p3_1 = gluon.nn.AvgPool2D(pool_size=3, strides=1, padding=1)
        self.p3_2 = Conv_BN_ReLU(n3, kernel_size=1)
        # path4
        self.p4_1 = Conv_BN_ReLU(n4, kernel_size=1)
        
    def forward(self, X):
        p1 = self.p1_2(self.p1_1(X))
        p1 = nd.concat(self.p1_3_1(p1), self.p1_3_2(p1), dim=1)
        
        p2 = self.p2_1(X)
        p2 = nd.concat(self.p2_2_1(p2), self.p2_2_2(p2), dim=1)
        
        p3 = self.p3_2(self.p3_1(X))
        
        p4 = self.p4_1(X)
        
        if self.debug:
            print("p1 : ", p1.shape)
            print("p2 : ", p2.shape)
            print("p3 : ", p3.shape)
            print("p4 : ", p4.shape)
        out = nd.concat(p1, p2, p3, p4, dim=1)
        return out
        

In [17]:
incep7 = Inception7([448, 384, [384, 384]], [384, [384,384]], 192, 320, debug=True)
incep7.initialize(ctx=ctx)
X = nd.random.uniform(shape=(1, 1280, 8, 8), ctx=ctx)
y = incep7(X)
y.shape 

p1 :  (1, 768, 8, 8)
p2 :  (1, 768, 8, 8)
p3 :  (1, 192, 8, 8)
p4 :  (1, 320, 8, 8)


(1, 2048, 8, 8)

## GoogLeNet V3

In [18]:
class GoogLeNetV3(gluon.Block):
    def __init__(self, num_classes, verbose=False, **kwargs):
        super().__init__(**kwargs)
        self.verbose = verbose
        with self.name_scope():
            b1 = gluon.nn.Sequential()
            b1.add(
                # 3 X 299 X 299
                Conv_BN_ReLU(channels=32, kernel_size=3, strides=2),
                # 32 X 149 X 149
                Conv_BN_ReLU(channels=32, kernel_size=3),
                # 32 X 147 X 147
                Conv_BN_ReLU(channels=64, kernel_size=3, padding=1),
                # 64 X 147 X 147
                gluon.nn.MaxPool2D(pool_size=3, strides=2),
                # 64 X 73 X 73
            )
            b2 = gluon.nn.Sequential()
            b2.add(
                # 64 X 73 X 73
                Conv_BN_ReLU(channels=80, kernel_size=3),
                # 80 X 71 X 71
                Conv_BN_ReLU(channels=192, kernel_size=3, strides=2),
                # 192 X 35 X 35
                Conv_BN_ReLU(channels=288, kernel_size=3, padding=1)
                # 288 X 35 X 35
            )
            b3 = gluon.nn.Sequential()
            b3.add(
                # 288 X 35 X 35
                Inception5(64, 48, 64, 64, 96, 96, 32),
                Inception5(64, 48, 64, 64, 96, 96, 64),
                Inception5(64, 48, 64, 64, 96, 96, 64),
                Inception5_1(384, 64, 96, 96),
            )
            b4 = gluon.nn.Sequential()
            b4.add(
                # 原论文中说使用5个，不知道为什么这边是4个 论文中使用的n=7
                Inception6(7, [128, 128, 128, 128, 192], [128, 128, 192], 192, 192),
                Inception6(7, [160, 160, 160, 160, 192], [160, 160, 192], 192, 192),
                Inception6(7, [160, 160, 160, 160, 192], [160, 160, 192], 192, 192),
                Inception6(7, [192, 192, 192, 192, 192], [192, 192, 192], 192, 192),
                Inception6_1(7, [192, 320], [192, 192, 192, 192])
            )
            b5 = gluon.nn.Sequential()
            b5.add(
                Inception7([448, 384, [384, 384]], [384, [384,384]], 192, 320),
                Inception7([448, 384, [384, 384]], [384, [384,384]], 192, 320),
                gluon.nn.AvgPool2D(pool_size=8, strides=1)
            )
            b6 = gluon.nn.Sequential()
            b6.add(
                gluon.nn.Flatten(),
                gluon.nn.Dense(num_classes)
            )
        self.net = gluon.nn.Sequential()
        self.net.add(b1, b2, b3, b4, b5, b6)
            
            
    def forward(self, X):
        out = X 
        for i, blk in enumerate(self.net):
            out = blk(out)
            if self.verbose:
                print("blk %d : %s" % ((i+1), out.shape))
        return out

In [ ]:
gln = GoogLeNetV3(100, verbose=True)
gln.initialize(ctx=ctx)
image = nd.random.uniform(shape=(1, 3, 299, 299), ctx=ctx)
y = gln(image)

blk 1 : (1, 64, 73, 73)
blk 2 : (1, 288, 35, 35)
blk 3 : (1, 768, 17, 17)
blk 4 : (1, 1280, 8, 8)
blk 5 : (1, 2048, 1, 1)
blk 6 : (1, 100)


In [ ]:
from time import time

batch_size = 32
train_data, test_data = utils.load_dataset(batch_size, resize=299, data_type='cifar10')

gln = GoogLeNetV3(num_classes=10, verbose=False)
gln.collect_params().initialize(mx.init.Xavier(), ctx=ctx, force_reinit=True)

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(gln.collect_params(), 'sgd', {'learning_rate' : 0.5})

epochs = 10

niter = 0
moving_loss = 0.0
smoothing_constant = 0.9

from time import time
for epoch in range(epochs):
    start = time()
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with autograd.record():
            output = gln(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)
        
        niter += 1
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = smoothing_constant * moving_loss + (1-smoothing_constant) * curr_loss
        estimated_loss = moving_loss / (1 - smoothing_constant**niter)
    
    train_acc = utils.evaluate_accuracy_gluon(train_data, gln, ctx)
    test_acc = utils.evaluate_accuracy_gluon(test_data, gln, ctx)
    print("Epoch %d, Moving Train Avg loss %.5f, Train acc %.5f, Test acc %.5f, Time consume %.5f s."
         % (epoch, estimated_loss, train_acc, test_acc, time() - start))

(32, 3, 299, 299)

[ 5.  4.  9.  3.  6.  2.  7.  6.  0.  0.  7.  3.  1.  5.  9.  0.  6.  8.
  0.  9.  5.  6.  0.  8.  7.  0.  8.  4.  7.  5.  7.  5.]
<NDArray 32 @cpu(0)>
Epoch 0, Moving Train Avg loss 1.49369, Train acc 0.48674, Test acc 0.48150, Time consume 698.66374 s.


## 补充结果

<img src="../img/Chapter4-Convolutional-Neural-Networks/4-14.png" width="800">